首先先定义出欧式距离计算函数

In [1]:
import numpy as np
def distEclu(arrA, arrB):
    # sum 用平方和代替距离
    dist = np.sum(arrA-arrB, axis=1)
    return dist

接着定义自动生成随机质心的函数，使用numpy.random.uniform()函数生成随机质心

In [4]:
def randCent(dataset, k):
     # dataset 导入数据 k 质心数 
    n = dataset.shape[1] # 特征有多少列
    data_min = dataset.iloc[:, :n-1].min() # 每一列的最小值;这个列不包含标签的为：n-1
    data_max = dataset.iloc[:, :n-1].max() # 每一列的最大值
    data_cent = np.random.uniform(data_min, data_max, (k, n-1)) # 参数：最小值,最大值,shape形状。
    # 返回生成的质心
    return data_cent

将两函数将加入kmeans代码类中

In [6]:
class KMeans:
    def __init__(self, k, max_iters=100, tol=1e-4):
        """
        初始化 KMeans 类

        参数:
        k: 聚类数量
        max_iters: 最大迭代次数
        tol: 容忍度，用于判断质心是否不再移动
        """
        self.k = k
        self.max_iters = max_iters
        self.tol = tol
        self.centroids = None
        self.labels = None

    def distEclu(self, arrA, arrB):
        """
        计算欧氏距离的平方和

        参数:
        arrA: 数据点
        arrB: 质心

        返回:
        欧氏距离的平方和
        """
        dist = np.sum((arrA - arrB) ** 2, axis=1)
        return dist
    # 找到每个点最近的质心
    def randCent(self, dataset):
        # 参数:dataset: 数据集
        n = dataset.shape[1]  # 特征有多少列
        data_min = dataset.iloc[:, :n - 1].min()  # 每一列的最小值;这个列不包含标签的为：n-1
        data_max = dataset.iloc[:, :n - 1].max()  # 每一列的最大值
        data_cent = np.random.uniform(data_min, data_max, (self.k, n - 1))  # 参数：最小值,最大值,shape形状。
        # 返回生成的质心
        return data_cent


向类中加入寻找最近质点和更新质点函数和结果函数

In [10]:
import numpy as np
import pandas as pd

class KMeans:
    def __init__(self, k, max_iters=100, tol=1e-4):
        """
        初始化 KMeans 类

        参数:
        k: 聚类数量
        max_iters: 最大迭代次数
        tol: 容忍度，用于判断质心是否不再移动
        """
        self.k = k
        self.max_iters = max_iters
        self.tol = tol
        self.centroids = None
        self.labels = None

    def distEclu(self, arrA, arrB):
        """
        计算欧氏距离的平方和

        参数:
        arrA: 数据点
        arrB: 质心

        返回:
        欧氏距离的平方和
        """
        dist = np.sum((arrA - arrB) ** 2, axis=1)
        return dist

    def randCent(self, dataset):
        """
        随机初始化质心

        参数:
        dataset: 数据集

        返回:
        初始化的质心
        """
        n = dataset.shape[1]  # 特征有多少列
        data_min = dataset.iloc[:, :n - 1].min()  # 每一列的最小值;这个列不包含标签的为：n-1
        data_max = dataset.iloc[:, :n - 1].max()  # 每一列的最大值
        data_cent = np.random.uniform(data_min, data_max, (self.k, n - 1))  # 参数：最小值,最大值,shape形状。
        # 返回生成的质心
        return data_cent

    def closest_centroid(self, X):
        """
        找到每个点最近的质心

        参数:
        X: 数据集

        返回:
        每个点最近的质心的索引
        """
        distances = np.array([self.distEclu(X, centroid) for centroid in self.centroids])
        return np.argmin(distances, axis=0)

    def update_centroids(self, X, labels):
        """
        更新质心位置

        参数:
        X: 数据集
        labels: 点的标签

        返回:
        新的质心位置
        """
        new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(self.k)])
        return new_centroids

    def fit(self, X):
        """
        执行 K-means 聚类

        参数:
        X: 数据集
        """
        self.centroids = self.randCent(X)
        for i in range(self.max_iters):
            self.labels = self.closest_centroid(X)
            new_centroids = self.update_centroids(X, self.labels)
            if np.all(np.abs(new_centroids - self.centroids) < self.tol):
                break
            self.centroids = new_centroids

    def predict(self, X):
        """
        预测数据点的簇

        参数:
        X: 数据集

        返回:
        数据点的簇标签
        """
        return self.closest_centroid(X)